まずAnaconda(https://www.anaconda.com/) をインストール。インストール方法はググれば出てきます。

Anacondaをインストールしたら、AnacondaPromptを起動して、$ pip install opencv-pythonを入力してOpenCVをインストールする。

その後、下のセルが実行できるか確かめる。

In [1]:
import cv2
import numpy as np
print(cv2.__version__)
print(np.__version__)

4.1.1
1.16.5


プログラムファイルが置いてあるディレクトリと同じ場所に動画ファイルを置いておく。wmv、mp4、aviは確実に対応しているっぽいです。

実行すると、ファイル名の入力を求められるのでファイル名を拡張子まで入力する。入力すると、元の動画と背景差分で動いているものだけを切り抜いた動画がウインドウに表示されて、同時に背景差分された動画が.aviで保存されます。処理が終わると自動で実行終了されますが、Escキーで抜ける。

In [2]:
import os
import cv2
import numpy as np

# 定数定義
ESC_KEY = 27     # Escキー
INTERVAL= 10     # インターバル
FRAME_RATE = 15  # fps

WINDOW_ORG = "orginal"
WINDOW_DIFF = "diff"

print("ファイル名を入力してください(拡張子を含む)")
FILE_ORG = input()
filename, ext = os.path.splitext(FILE_ORG)

# ウィンドウの準備
cv2.namedWindow(WINDOW_ORG, cv2.WINDOW_NORMAL)
cv2.namedWindow(WINDOW_DIFF, cv2.WINDOW_NORMAL)

# 元ビデオファイル読み込み
mov_org = cv2.VideoCapture(FILE_ORG)

# 比較用の最初のフレーム読み込み
ret, frame = mov_org.read()

# 背景フレーム
back_frame = np.zeros_like(frame, np.float32)

# 出力動画のコーデック設定
width = int(mov_org.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(mov_org.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = mov_org.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
mov_out = cv2.VideoWriter(filename + '_output.avi', fourcc, fps, (width,height))

# 変換処理ループ
while ret == True:
    # 入力画像を浮動小数点型に変換
    flt32_frame = frame.astype(np.float32)

    # 差分計算
    diff_frame = cv2.absdiff(flt32_frame, back_frame)

    # 背景の更新
    cv2.accumulateWeighted(flt32_frame, back_frame, 0.1)

    # フレーム表示
    cv2.imshow(WINDOW_ORG, frame)
    cv2.imshow(WINDOW_DIFF, diff_frame.astype(np.uint8))

    # 背景差分フレームを動画に出力
    mov_out.write(diff_frame.astype(np.uint8))
    
    # Escキーで終了
    key = cv2.waitKey(INTERVAL)
    if key == ESC_KEY:
        break

    # 次のフレーム読み込み
    ret, frame = mov_org.read()

# 終了処理
cv2.destroyAllWindows()
mov_org.release()
mov_out.release()

ファイル名を入力してください(拡張子を含む)
2.wmv
